# Submission line generator
Generate the lenghty submission line for running hparam optimization from a CSV containing planned experiments.

In [ ]:
import pandas as pd

In [ ]:
# read db export
df = pd.read_csv("exp_db.csv").dropna(how="all")
# select where to start and stop generating
df = df.loc[df["Short name"].apply(lambda x: 1799 < int(x[2:]) < 1810)].sort_values(by=["Short name"])
df

In [ ]:
def get_features(s):
    if s == "CGR":
        return "None"
    s = s.replace(",", "")
    s = s.replace("CGR", "")
    s = s.strip()
    return s
        

In [ ]:
for i, row in df.iterrows():
    model = row["Estimator"].replace("-", "").replace("Logistic Regression", "LogReg")
    data_path = f'data/{row["Data set ID"]}.csv'
    global_features = get_features(row["Features"])
    exp_id = row["Short name"]
    split = row["Split ID"]
    #tags = "_".join([row["Split ID"].split("_")[-3], row["Split ID"].split("_")[-1], "syn"])  # for "0D_80_syn" style
    #tags = "_".join([row["Split ID"].split("_")[-3], row["Split ID"].split("_")[-1]])  # for "0D_80" style
    tags = row["Split ID"].split("_")[-2] # for "1D" style
    # for workstation
    #print(f"nq /home/julian/miniconda3/envs/synferm-predictions/bin/python /home/julian/PycharmProjects/synferm-predictions/run.py train --config config/config_{model}.yaml --data-path {data_path} --split-indices data/splits/{split}/ --smiles-columns I_smiles M_smiles T_smiles --label-columns binary_A binary_B binary_C --global-features {global_features} --global-features-file data/4D_features_smiles.json --task multilabel --run-test --hparam-optimization --hparam-config-path config/hparam_bounds_{model}.yaml --hparam-n-iter 20 --experiment-id {exp_id} --tags {tags}")
    # for cluster
    print(f'sbatch -n 1 --cpus-per-task=4 --time=4:00:00 --mem-per-cpu=1024 --mail-type=END --wrap="/cluster/home/jgoetz/miniconda3/envs/synferm-predictions/bin/python /cluster/home/jgoetz/synferm-predictions/run.py train --config config/config_{model}.yaml --data-path {data_path} --split-indices data/splits/{split}/ --smiles-columns I_smiles M_smiles T_smiles --label-columns binary_A binary_B binary_C --global-features {global_features} --global-features-file data/4D_features_smiles.json --task multilabel --run-test --hparam-optimization --hparam-config-path config/hparam_bounds_{model}.yaml --hparam-n-iter 20 --experiment-id {exp_id} --tags {tags} --wandb-offline"')
    print()
